# Task 1

Y - student will find XAI useful in future \
$\hat{Y}$ - student enrolled in training \
A - population \

#Demographic parity

P($\hat{Y}$ = 1|A = blue) = 0.65 \
P($\hat{Y}$ = 1|A = red) = 0.5 \
0.65/0.5 = 1.3

#Equal opportunity

P($\hat{Y}$ = 1|A = blue, Y = 1) = 0.75 \
P($\hat{Y}$ = 1|A = red, Y = 1) = 0.5 \
0.75/0.5 = 1.5

# Predictive rate parity

P(Y = 1|A = blue, $\hat{Y}$ = 1) = $\dfrac{12}{13}$ \
P(Y = 1|A = red, $\hat{Y}$ = 1) = 0.5 \
12/13 * 2 = $\dfrac{24}{13}$

# How can this decision rule be changed to improve its fairness?

Blue population is privleged compare to red one. If we will do some selection for blue population to enroll the course so that less people from this group will do it we would improve the fairness.

# Task 2

In [2]:
import dalex as dx
import xgboost

import sklearn
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import platform
print(f'Python {platform.python_version()}')

{package.__name__: package.__version__ for package in [dx, xgboost, sklearn, pd, np]}

Python 3.8.15


{'dalex': '1.5.0',
 'xgboost': '1.7.1',
 'sklearn': '1.0.2',
 'pandas': '1.3.5',
 'numpy': '1.21.6'}

# Lets load, clean and see the data

In [3]:
df = pd.read_csv('adult.csv')

df = df.dropna()

df.loc[:, df.dtypes == 'object'] =\
    df.select_dtypes(['object'])\
    .apply(lambda x: x.astype('category'))


df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [4]:
X = df.drop(columns = 'income')
 
y = df.income
y.replace({'<=50K':0,'>50K':1},inplace=True)

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.33, random_state=42)

y

0        0
1        0
2        1
3        1
4        0
        ..
48837    0
48838    1
48839    0
48840    0
48841    1
Name: income, Length: 48842, dtype: int64

Now lets make data for diffrent model. I will do feature transformation by multiply column "educational-num" and "hours-per-week". I believe it is nice idea because we can image if someone spent more time on education his work will be more valuable.

In [5]:
df1 = pd.read_csv('adult.csv')

df1 = df1.dropna()

df1.loc[:, df1.dtypes == 'object'] =\
    df1.select_dtypes(['object'])\
    .apply(lambda x: x.astype('category'))

df1['weaight-hours'] = df1['educational-num']*df1['hours-per-week']

X1 = df1.drop(columns = 'income')
 
y1 = df1.income
y1.replace({'<=50K':0,'>50K':1},inplace=True)

X1_train, X1_test, y1_train, y1_test = sklearn.model_selection.train_test_split(X1, y1, test_size=0.33, random_state=42)

In [6]:
X_train.dtypes

age                   int64
workclass          category
fnlwgt                int64
education          category
educational-num       int64
marital-status     category
occupation         category
relationship       category
race               category
gender             category
capital-gain          int64
capital-loss          int64
hours-per-week        int64
native-country     category
dtype: object

# Now it is time for model training

In [7]:
model = xgboost.XGBClassifier(
    n_estimators=50, 
    max_depth=2, 
    use_label_encoder=False, 
    eval_metric="logloss",
    
    enable_categorical=True,
    tree_method="hist"
)

model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=True,
              eval_metric='logloss', feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=2, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=50, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [8]:
model1 = xgboost.XGBClassifier(
    n_estimators=60, 
    max_depth=3, 
    use_label_encoder=False, 
    eval_metric="logloss",
    
    enable_categorical=True,
    tree_method="hist"
)

model1.fit(X1_train, y1_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=True,
              eval_metric='logloss', feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=60, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, ...)

Lets define explaners and see thier performences

In [9]:
def pf_xgboost_classifier_categorical(model, df):
    df.loc[:, df.dtypes == 'object'] =\
        df.select_dtypes(['object'])\
        .apply(lambda x: x.astype('category'))
    return model.predict_proba(df)[:, 1]

explainer = dx.Explainer(model, X_test, y_test, predict_function=pf_xgboost_classifier_categorical)

Preparation of a new explainer is initiated

  -> data              : 16118 rows 14 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 16118 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function pf_xgboost_classifier_categorical at 0x7ff14bd86790> will be used
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.000607, mean = 0.238, max = 0.996
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.98, mean = -0.00251, max = 0.994
  -> model_info        : package xgboost

A new explainer has been created!


In [10]:
explainer1 = dx.Explainer(model1, X1_test, y1_test, predict_function=pf_xgboost_classifier_categorical)

Preparation of a new explainer is initiated

  -> data              : 16118 rows 15 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 16118 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function pf_xgboost_classifier_categorical at 0x7ff14bd86790> will be used
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.000118, mean = 0.238, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.996, mean = -0.0022, max = 0.997
  -> model_info        : package xgboost

A new explainer has been created!


In [11]:
explainer.model_performance()

,recall,precision,f1,accuracy,auc
XGBClassifier,0.619674,0.789544,0.694371,0.871324,0.924449


In [12]:
explainer1.model_performance()

,recall,precision,f1,accuracy,auc
XGBClassifier,0.652551,0.78837,0.71406,0.876722,0.92929


Now let's count fairness and check it.

In [13]:
protected_variable = X_test['race']
privileged_group = "White"

fobject = explainer.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)

In [14]:
protected_variable1 = X1_test['race']
privileged_group = "White"

fobject1 = explainer1.model_fairness(
    protected=protected_variable1,
    privileged=privileged_group
)

In [15]:
fobject.fairness_check()

Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'White'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
                         TPR       ACC       PPV       FPR       STP
Amer-Indian-Eskimo  0.801282  1.090278  0.981084  0.254545  0.291457
Asian-Pac-Islander  1.099359  0.987269  0.848676  1.763636  1.160804
Black               0.822115  1.074074  1.030265  0.290909  0.376884
Other               0.942308  1.077546  0.969735  0.436364  0.452261


for this model FPR and STP metrics are out of scale, so model is unfair - it is bais towards White people.

In [16]:
fobject1.fairness_check()

Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'White'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
                         TPR       ACC       PPV       FPR       STP
Amer-Indian-Eskimo  0.762195  1.084005  0.984810  0.237288  0.276190
Asian-Pac-Islander  1.146341  1.000000  0.877215  1.644068  1.171429
Black               0.830793  1.072497  1.044304  0.271186  0.376190
Other               0.896341  1.079402  1.054430  0.271186  0.395238


in this case TPR, FPR and STP metrics are out of scale, so model is even more unfair - it is bais towards White people.

In [17]:
fobject.plot()

In [18]:
fobject1.plot()


Found NaN's or 0's for models: {'XGBClassifier'}
It is advisable to check 'metric_ratios'


# Droping "race" column to avoid biases
As we can see our models are unfair. We can try to avoid it by droping "race" column.

In [19]:
X_train_without_prot, X_test_without_prot = X_train.drop("race", axis=1), X_test.drop("race", axis=1)

model_without_prot = xgboost.XGBClassifier(
    n_estimators=50, 
    max_depth=2, 
    use_label_encoder=False, 
    eval_metric="logloss",
    enable_categorical=True,
    tree_method="hist"
)

model_without_prot.fit(X_train_without_prot, y_train)

explainer_without_prot = dx.Explainer(
    model_without_prot, 
    X_test_without_prot, 
    y_test,
    predict_function=pf_xgboost_classifier_categorical,
    label="XGBClassifier without the protected attribute",
    verbose=False
)

fobject_without_prot = explainer_without_prot.model_fairness(protected_variable, privileged_group)

In [20]:
X1_train_without_prot, X1_test_without_prot = X1_train.drop("race", axis=1), X1_test.drop("race", axis=1)

model1_without_prot = xgboost.XGBClassifier(
    n_estimators=60, 
    max_depth=3, 
    use_label_encoder=False, 
    eval_metric="logloss",
    enable_categorical=True,
    tree_method="hist"
)

model1_without_prot.fit(X1_train_without_prot, y1_train)

explainer1_without_prot = dx.Explainer(
    model1_without_prot, 
    X1_test_without_prot, 
    y1_test,
    predict_function=pf_xgboost_classifier_categorical,
    label="XGBClassifier without the protected attribute",
    verbose=False
)

fobject1_without_prot = explainer1_without_prot.model_fairness(protected_variable1, privileged_group)

As we can see below it did not help.

In [21]:
fobject.plot(fobject_without_prot, show=False).\
    update_layout(autosize=False, width=800, height=450, legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99))

In [22]:
fobject1.plot(fobject1_without_prot, show=False).\
    update_layout(autosize=False, width=800, height=450, legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99))


Found NaN's or 0's for models: {'XGBClassifier'}
It is advisable to check 'metric_ratios'


# Try to make models fair by using "roc pivot"

In [23]:
from dalex.fairness import resample, reweight, roc_pivot
from copy import copy

protected_variable_train = X_train['race']

# roc_pivot
explainer_roc_pivot = roc_pivot(
    copy(explainer_without_prot), 
    protected_variable, 
    privileged_group,
    verbose=False
)
explainer_roc_pivot.label = 'XGBClassifier with ROC pivot mitigation'
fobject_roc_pivot = explainer_roc_pivot.model_fairness(
    protected_variable, 
    privileged_group
)

In [24]:
from dalex.fairness import resample, reweight, roc_pivot
from copy import copy

protected1_variable_train = X_train['race']

# roc_pivot
explainer1_roc_pivot = roc_pivot(
    copy(explainer1_without_prot), 
    protected_variable1, 
    privileged_group,
    verbose=False
)
explainer1_roc_pivot.label = 'XGBClassifier with ROC pivot mitigation'
fobject1_roc_pivot = explainer1_roc_pivot.model_fairness(
    protected_variable1, 
    privileged_group
)

In [25]:
fobject_without_prot.plot([fobject_roc_pivot], show=False).\
    update_layout(autosize=False, width=800, height=450, legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99))

In [26]:
fobject1_without_prot.plot([fobject1_roc_pivot], show=False).\
    update_layout(autosize=False, width=800, height=450, legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99))

As we can see above and below it did not help.

In [27]:
print("\n========== " + fobject_roc_pivot.label + " ==========")
fobject_roc_pivot.fairness_check(epsilon=0.66)


========== XGBClassifier with ROC pivot mitigation ==========
Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'White'. Parameter 'epsilon' was set to 0.66 and therefore metrics should be within (0.66, 1.515)
                         TPR       ACC       PPV       FPR       STP
Amer-Indian-Eskimo  0.981100  1.085847  0.812424  0.651163  0.430168
Asian-Pac-Islander  1.243986  0.980278  0.778319  2.790698  1.430168
Black               0.989691  1.077726  0.943971  0.534884  0.497207
Other               1.111684  1.088167  0.957369  0.558140  0.541899


FPR and STP metrics are out of scale for all groups.

In [28]:
print("\n========== " + fobject1_roc_pivot.label + " ==========")
fobject1_roc_pivot.fairness_check(epsilon=0.66)


========== XGBClassifier with ROC pivot mitigation ==========
Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'White'. Parameter 'epsilon' was set to 0.66 and therefore metrics should be within (0.66, 1.515)
                         TPR       ACC       PPV       FPR       STP
Amer-Indian-Eskimo  0.821018  1.071511  0.777506  0.608696  0.377660
Asian-Pac-Islander  1.188834  0.981546  0.800733  2.413043  1.329787
Black               1.031199  1.073818  0.916870  0.608696  0.531915
Other               1.062397  1.088812  1.034230  0.347826  0.478723


For this model FPR and STP metrics are  even more out of scale for all groups.

In [29]:
pd.concat([
    explainer_without_prot.model_performance().result, 
    explainer_roc_pivot.model_performance().result
], axis=0)

,recall,precision,f1,accuracy,auc
XGBClassifier without the protected attribute,0.619411,0.791863,0.695100,0.871820,0.924374
XGBClassifier with ROC pivot mitigation,0.586007,0.809887,0.679994,0.869897,0.924210


In [30]:
pd.concat([
    explainer1_without_prot.model_performance().result, 
    explainer1_roc_pivot.model_performance().result
], axis=0)

,recall,precision,f1,accuracy,auc
XGBClassifier without the protected attribute,0.648080,0.787472,0.711009,0.875729,0.929159
XGBClassifier with ROC pivot mitigation,0.612572,0.807279,0.696575,0.874116,0.929068


# summary
I coud not made models fair. Feature transformation aproved recall and a little bit accuracy but it made model more unfair.